In [1]:
import requests
import pickle
import pandas as pd
import json as js

In [2]:
game_data = []
year = '2019'
season_type = '02' 
max_game_ID = 1290

In [3]:
for i in range(1083,max_game_ID):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(i).zfill(4)+'/feed/live')
    data = r.json()
    game_data.append(data)

In [28]:
#ezzel atugorjuk a 0-s jatekot 
#game_data = game_data[1:]

In [4]:
#lista elso elemet vizsgaljuk
game_data[0]

{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2022. All Rights Reserved.',
 'gamePk': 2019021083,
 'link': '/api/v1/game/2019021083/feed/live',
 'metaData': {'wait': 10, 'timeStamp': '20200312_174149'},
 'gameData': {'game': {'pk': 2019021083, 'season': '20192020', 'type': 'R'},
  'datetime': {'dateTime': '2020-03-12T23:00:00Z'},
  'status': {'abstractGameState': 'Preview',
   'codedGameState': '9',
   'detailedState': 'Postponed',
   'statusCode': '9',
   'startTimeTBD': False},
  'teams': {'away': {'id': 18,
    'name': 'Nashville Predators',
    'link': '/api/v1/teams/18',
    'venue': {'id': 5030,
     'name': 'Bridgestone Arena',
     'link': '/api/v1/venues/5030',
     'city': 'Nashville',
     'timeZone': {'id': 'America/Chicago', 'offset': -5, 'tz': 'CDT'}},
    'abbreviation': 'NSH',
    'triCode': 'NSH',
    'teamName': 'Predators',
    'locationName': 'Nas

In [60]:
len(game_data)

207

In [5]:
players = list()

for idx, i in enumerate(game_data):
    try:
        tmp_players_df = pd.DataFrame(i['gameData']['players']).transpose()
        tmp_players_df = tmp_players_df.assign(gameId = i['gameData']['game']['pk'])
        tmp_players_df = tmp_players_df.assign(teamID = lambda x: x['currentTeam'].apply(lambda y: y['id']))
        tmp_players_df = tmp_players_df.assign(pPos = lambda x: x['primaryPosition'].apply(lambda y: y['name']))
        tmp_players_df = tmp_players_df.drop(columns=['currentTeam', 'primaryPosition'])
        players.append(tmp_players_df)
    except:
        continue

players = pd.concat(players)
players.to_csv('players_df_1290.csv', index=False)

players

ValueError: No objects to concatenate

In [50]:
teams = list()

for i in game_data:
    try:
        tmp_teams_df = pd.DataFrame(i['gameData']['teams']).transpose()
        tmp_teams_df = tmp_teams_df.drop(columns=['venue','division','conference','franchise','officialSiteUrl','franchiseId'])
        teams.append(tmp_teams_df)
    except:
        continue

teams = pd.concat(teams)
teams.to_csv('teams_df_1082.csv', index=False)
teams

,id,name,link,abbreviation,triCode,teamName,locationName,firstYearOfPlay,shortName,active
away,29,Columbus Blue Jackets,/api/v1/teams/29,CBJ,CBJ,Blue Jackets,Columbus,2000,Columbus,True
home,9,Ottawa Senators,/api/v1/teams/9,OTT,OTT,Senators,Ottawa,1992,Ottawa,True
away,7,Buffalo Sabres,/api/v1/teams/7,BUF,BUF,Sabres,Buffalo,1970,Buffalo,True
home,2,New York Islanders,/api/v1/teams/2,NYI,NYI,Islanders,New York,1972,NY Islanders,True
away,12,Carolina Hurricanes,/api/v1/teams/12,CAR,CAR,Hurricanes,Carolina,1997,Carolina,True
...,...,...,...,...,...,...,...,...,...,...
home,18,Nashville Predators,/api/v1/teams/18,NSH,NSH,Predators,Nashville,1998,Nashville,True
away,22,Edmonton Oilers,/api/v1/teams/22,EDM,EDM,Oilers,Edmonton,1979,Edmonton,True
home,20,Calgary Flames,/api/v1/teams/20,CGY,CGY,Flames,Calgary,1980,Calgary,True
away,54,Vegas Golden Knights,/api/v1/teams/54,VGK,VGK,Golden Knights,Vegas,2017,Vegas,True


In [55]:
games = list()

for idx, i in enumerate(game_data):
    try:
        tmp_games_df = pd.DataFrame(i['gameData']['game'],index=[0])
        tmp_games_df = tmp_games_df.assign(startDate = i['gameData']['datetime']['dateTime'])
        tmp_games_df = tmp_games_df.assign(endDate = i['gameData']['datetime']['endDateTime'])
        tmp_games_df = tmp_games_df.assign(status = i['gameData']['status']['abstractGameState'])
        tmp_games_df = tmp_games_df.assign(teamAwayID = i['gameData']['teams']['away']['id'])
        tmp_games_df = tmp_games_df.assign(teamHomeID = i['gameData']['teams']['home']['id'])
        games.append(tmp_games_df)
    except:
        continue
            
games = pd.concat(games)
games.to_csv('games_df_1290.csv', index=False)
    
games

ValueError: No objects to concatenate

In [48]:
events = list()

for idx, i in enumerate(game_data):
    try:
        tmp_events_df = pd.DataFrame(i['liveData']['plays']['allPlays'])
        tmp_events_df = tmp_events_df.assign(gameId = i['gameData']['game']['pk'])
        tmp_events_df = tmp_events_df.assign(eventId = lambda x: x['about'].apply(lambda y: y['eventIdx']))
        tmp_events_df = tmp_events_df.assign(eventType = lambda x: x['result'].apply(lambda y: y['event']))
        tmp_events_df = tmp_events_df.assign(eventTypeDesc = lambda x: x['result'].apply(lambda y: y['description']))
        tmp_events_df = tmp_events_df.assign(period = lambda x: x['about'].apply(lambda y: y['period']))
        tmp_events_df = tmp_events_df.assign(periodTime = lambda x: x['about'].apply(lambda y: y['periodTime']))
        tmp_events_df = tmp_events_df.assign(periodType = lambda x: x['about'].apply(lambda y: y['periodType']))
        tmp_events_df = tmp_events_df.assign(goalsHome = lambda x: x['about'].apply(lambda y: y['goals']['home']))
        tmp_events_df = tmp_events_df.assign(goalsAway = lambda x: x['about'].apply(lambda y: y['goals']['away']))
        tmp_events_df = tmp_events_df.assign(coX = lambda x: x['coordinates'].apply(lambda y: y['x'] if y else ''))
        tmp_events_df = tmp_events_df.assign(coY = lambda x: x['coordinates'].apply(lambda y: y['y'] if y else ''))

        result_player1Id = []
        result_player1Type = []

        for i in tmp_events_df['players']:
            try:
                result_player1Id.append(i[0]['player']['id'])
                result_player1Type.append(i[0]['playerType'])
            except IndexError:
                result_player1Id.append("")
                result_player1Type.append("")
            except TypeError:
                result_player1Id.append('')
                result_player1Type.append("")
            except ValueError:
                result_player1Id.append('')
                result_player1Type.append("")     

        result_player2Id = []
        result_player2Type = []

        for i in tmp_events_df['players']:
            try:
                result_player2Id.append(i[1]['player']['id'])
                result_player2Type.append(i[1]['playerType'])
            except IndexError:
                result_player2Id.append('')
                result_player2Type.append('')
            except TypeError:
                result_player2Id.append('')
                result_player2Type.append('')
            except ValueError:
                result_player2Id.append('')
                result_player2Type.append('')

        result_player3Id = []
        result_player3Type = []

        for i in tmp_events_df['players']:
            try:
                result_player3Id.append(i[2]['player']['id'])
                result_player3Type.append(i[2]['playerType'])
            except IndexError:
                result_player3Id.append('')
                result_player3Type.append('')
            except TypeError:
                result_player3Id.append('')
                result_player3Type.append('')
            except ValueError:
                result_player3Id.append('')
                result_player3Type.append('')

        result_player4Id = []
        result_player4Type = []

        for i in tmp_events_df['players']:
            try:
                result_player4Id.append(i[3]['player']['id'])
                result_player4Type.append(i[3]['playerType'])
            except IndexError:
                result_player4Id.append('')
                result_player4Type.append('')
            except TypeError:
                result_player4Id.append('')
                result_player4Type.append('')
            except ValueError:
                result_player4Id.append('')
                result_player4Type.append('')

        tmp_events_df = tmp_events_df.assign(player1Id = result_player1Id)
        tmp_events_df = tmp_events_df.assign(player1Type = result_player1Type)
        tmp_events_df = tmp_events_df.assign(player2Id = result_player2Id)
        tmp_events_df = tmp_events_df.assign(player2Type = result_player2Type)
        tmp_events_df = tmp_events_df.assign(player3Id = result_player3Id)
        tmp_events_df = tmp_events_df.assign(player3Type = result_player3Type)
        tmp_events_df = tmp_events_df.assign(player4Id = result_player4Id)
        tmp_events_df = tmp_events_df.assign(player4Type = result_player4Type)

        tmp_events_df = tmp_events_df.drop(columns=(['result','about','coordinates','players','team']))
            
        events.append(tmp_events_df)
    except:
        continue
    
events = pd.concat(events)
events.to_csv('events_df_1082.csv', index=False)

events

,gameId,eventId,eventType,eventTypeDesc,period,periodTime,periodType,goalsHome,goalsAway,coX,coY,player1Id,player1Type,player2Id,player2Type,player3Id,player3Type,player4Id,player4Type
0,2019020501,0,Game Scheduled,Game Scheduled,1,00:00,REGULAR,0,0,,,,,,,,,,
1,2019020501,1,Period Ready,Period Ready,1,00:00,REGULAR,0,0,,,,,,,,,,
2,2019020501,2,Period Start,Period Start,1,00:00,REGULAR,0,0,,,,,,,,,,
3,2019020501,3,Faceoff,Jean-Gabriel Pageau faceoff won against Pierre...,1,00:00,REGULAR,0,0,0.0,0.0,8476419,Winner,8479400,Loser,,,,
4,2019020501,4,Giveaway,Giveaway by Anders Nilsson,1,00:16,REGULAR,0,0,-94.0,-4.0,8475195,PlayerID,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,2019021082,303,Blocked Shot,Martin Frk shot blocked shot by Anthony Duclair,3,19:47,REGULAR,3,2,-51.0,-27.0,8477407,Blocker,8476924,Shooter,,,,
304,2019021082,304,Period End,End of 3rd Period,3,20:00,REGULAR,3,2,,,,,,,,,,
305,2019021082,305,Period Official,Period Official,3,20:00,REGULAR,3,2,,,,,,,,,,
306,2019021082,306,Game End,Game End,3,20:00,REGULAR,3,2,,,,,,,,,,
